# Step 4 update
Notebook to update step 4 of the work flow so it's clearner and faster

In [29]:
import pandas as pd
import numpy as np
import xarray as xr
from random import random
from itertools import groupby
from operator import itemgetter
import geopandas as gpd 
import glob
from statistics import mean
import julian
import time 
import multiprocessing as mp 
from multiprocessing import Pool
import os

In [7]:
## Arges Needed 
DATA_IN = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/HI/' # output from avg temp
DATA_OUT = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/'
dir_path = DATA_IN 
space_dim = 'ID_HDC_G0'
time_dim = 'date'
Tthresh = 40.6
fn_out = DATA_OUT+'STATS_1DAY406.json'

## Step 1

In [8]:
#### Step 1 - Function Loads all HImax Data as an X-array
def read_data(dir_path, space_dim, time_dim):
    """ Function reads in all HImax .csv files, joins them by date along the x-axis
    and returns the whole record as a x-array data array
    
    Args:   
        dir_path = path to .csv files 
        time_dim = name for time dim as a str ... use date :-)
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0)
    """
    fn_list = sorted(glob.glob(dir_path+'*.csv'))
    df_out = pd.DataFrame()
    date_list = []

    # Open all HImax files and concat into a df
    for i, fn in enumerate(fn_list):    
        # Open the CSV
        df = pd.read_csv(fn)

        # Get the city ids 
        if i == 1:
            df_id = df[space_dim]

        # get only the HImax columns and concate date list 
        df_temp = df.iloc[:,3:] # get only temp columns
        date_list = date_list+list(df_temp.columns)

        # Drop cities w/ no temp record 
        df_temp_drop = df_temp.dropna()

        # Merge
        df_out = pd.concat([df_out, df_temp_drop], axis=1)
        print(df_out.shape)
    
    # make date into an array
    HImax_arr = df_out.to_numpy()

    # Make data into an xr.DataArray
    HImax_xr_da = xr.DataArray(HImax_arr, coords=[df_id, date_list], 
                             dims=[space_dim, time_dim])
    return HImax_xr_da

In [9]:
HImax_xr_da = read_data(dir_path, space_dim, time_dim)

(13067, 365)
(13067, 731)
(13067, 1096)
(13067, 1461)
(13067, 1826)
(13067, 2192)
(13067, 2557)
(13067, 2922)
(13067, 3287)
(13067, 3653)
(13067, 4018)
(13067, 4383)
(13067, 4748)
(13067, 5114)
(13067, 5479)
(13067, 5844)
(13067, 6209)
(13067, 6575)
(13067, 6940)
(13067, 7305)
(13067, 7670)
(13067, 8036)
(13067, 8401)
(13067, 8766)
(13067, 9131)
(13067, 9497)
(13067, 9862)
(13067, 10227)
(13067, 10592)
(13067, 10958)
(13067, 11323)
(13067, 11688)
(13067, 12053)
(13067, 12419)


In [10]:
HImax_xr_da

<xarray.DataArray (ID_HDC_G0: 13067, date: 12419)>
array([[-49.590861, -38.042995, -36.750948, ..., -15.519086, -16.618087,
        -31.55629 ],
       [ -4.088565,  -2.252119,  -6.184317, ...,  -2.852921,  -2.796071,
         -2.456939],
       [-38.693309, -23.243517, -13.361891, ...,  -2.426149, -20.636527,
        -33.891277],
       ...,
       [ 15.595635,  16.668566,  23.193257, ...,  13.770282,  13.633869,
         16.228866],
       [ 16.370013,  16.564513,  16.169268, ...,  12.788926,  14.696112,
         12.831937],
       [ 16.383207,  16.337978,  20.977421, ...,   9.744847,  12.727527,
         13.662151]])
Coordinates:
  * ID_HDC_G0  (ID_HDC_G0) int64 5782 3316 5645 3185 ... 1116 1114 1161 1169
  * date       (date) <U10 '1983.01.01' '1983.01.02' ... '2016.12.31'

## Step 2

In [11]:
#### Step 2 Function finds all the Tmax Events and writes it to a dateframe w/ dates for each city
def tmax_days(xarray, Tthresh):
    """ Function finds all the tmax days in a year and sums total days per year 
    greater than a threshold within a year where Tmax > Tthresh for each city. Returns the total number of days,
    the dates, the tempatures, and the intensity (daily Tmax - Tthresh)
    
    Args: 
        xarray = an xarray object with dims = (space, times)
        Tthresh = int of temp threshold
    """
    
    # empty lists & df
    id_list = []
    date_list = []
    tmax_list = []
    intensity_list = []
    df_out = pd.DataFrame()
    
    # subset xarray
    out = xarray.where(xarray > Tthresh, drop = True)

    # start loop 
    for index, loc in enumerate(out.ID_HDC_G0):
        id_list.append(out.ID_HDC_G0.values[index]) # get IDS
        date_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values) # get event dates
        
        # #CPT 2020.02.23 
        # dayTot_list.append(len(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values)) # get event totals
        
        tmax_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values) # get temp values
        intensity_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values - Tthresh) # get severity

    # write to a data frame
    df_out['ID_HDC_G0'] = id_list
    # df_out['total_days'] = dayTot_list #CPT 2020.02.23
    df_out['dates'] = date_list
    df_out['tmax'] = tmax_list
    df_out['tmax_tntensity'] = intensity_list

    # return df_out
    return df_out

In [12]:
df_out = tmax_days(HImax_xr_da, Tthresh)

In [14]:
df_out.head()

,ID_HDC_G0,dates,tmax,tmax_tntensity
0,3554,[2010.07.29],[42.91095336309989],[2.310953363099891]
1,3295,[2010.07.29],[42.365039604142964],[1.765039604142963]
2,3242,[2010.07.29],[40.941567828562654],[0.34156782856265266]
3,9075,"[1988.07.22, 1988.07.23, 1988.07.24, 1988.07.2...","[42.60064411402778, 44.46700911035864, 44.7843...","[2.0006441140277786, 3.8670091103586373, 4.184..."
4,4362,"[1989.07.19, 1990.07.10, 1996.07.13, 2001.07.2...","[40.77021376234568, 42.44143213521773, 41.0918...","[0.17021376234568208, 1.8414321352177296, 0.49..."


## New: try chunking the DF

In [20]:
cpu = 4
n = int(len(df_out) / cpu) # number of CPU cores


In [22]:
list_df = [df_out[i:i+n] for i in range(0,df_out.shape[0],n)]

# Step 3

In [30]:
#### Step 3 Function splits the dataset into Tmax events (continuous days >Tmax) for each city
def jul_convert(dates):
    "Function turn days into julian datetime"
    jul_days = pd.to_datetime(dates).to_julian_date()
    
    return jul_days

def event_split(dates, ID_HDC_G0, intensity, tmax): #, total_days): #CPT 2020.02.23
    
    """ Searchs a list of dates and isolates sequential dates as a list, then calculates event stats.
    See comments in code for more details. 
    
    Args:
        dates: pandas.core.index as julian dates
        ID_HDC_G0: city ID as string
        intensity: numpy.ndarray of intensities values
        tmax: numpy.ndarray of intensities values of tmax values
        total_days: total number of tmax days in a year for a given city
    """

    # city id
    city_id = ID_HDC_G0
    # tot_days = total_days #CPT 2020.02.23
    
    # lists to fill
    city_id_list = []
    # tot_days_list = [] #CPT 2020.02.23
    event_dates_list = []
    dur_list = []
    intensity_list = []
    tmax_list = []
    avg_temp_list = []
    avg_int_list = []
    tot_int_list = []
    year_list = []
    
    # data frame out
    df_out = pd.DataFrame()
    
    # turn days into julian days
    jul_days = jul_convert(dates)
    
    # Counters to make sure we write the correct event dates to a list, don't want julian days in output
    counter = 0
    start = 0
    end = 0
    
    # Loop through dur list and isolate seq days, temps, and intensities
    for k, g in groupby(enumerate(jul_days.values), lambda x: x[1]-x[0]):
        
        seq = list(map(itemgetter(1), g)) # isolate seq. days
        dur = len(seq) # duration of each event
        
        counter = counter + dur # add duration to counter
        end = counter # end of current event
        
        event_dates = dates[start:end] # dates of tmax days during each event
        intense = intensity[start:end] # intensity of each day during event
        temp = tmax[start:end] # temp of each day during event
        avg_temp = mean(temp) # avg. temp during event
        avg_int = mean(intense) # avg. intensity during event
        tot_int = intense.sum() # total intensity during event
        
        start = counter # reset start to current end (e.g. counter)
        year = event_dates[0].split('.')[0]
        
        # fill lists
        city_id_list.append(city_id)
        year_list.append(year)
        # tot_days_list.append(tot_days) #CPT 2020.02.23
        dur_list.append(dur)
        event_dates_list.append(event_dates)
        intensity_list.append(intense)
        tmax_list.append(temp)
        avg_temp_list.append(avg_temp)
        avg_int_list.append(avg_int)
        tot_int_list.append(tot_int)

    # write out as a dateframe
    df_out['ID_HDC_G0'] = city_id_list
    df_out['year'] = year_list
    # df_out['total_days'] = tot_days_list #CPT 2020.02.23
    df_out['duration'] = dur_list
    df_out['avg_temp'] = avg_temp_list
    df_out['avg_intensity'] = avg_int_list
    df_out['tot_intensity'] = tot_int_list
    df_out['event_dates'] = event_dates_list
    df_out['duration'] = dur_list
    df_out['intensity'] = intensity_list
    df_out['tmax'] = tmax_list

    return df_out

#### Step 4 function feeds output from function 3 into function 4
def tmax_stats(df_in):
    """ runs event_split functionon a dataframe to produce desired tmax stats
        NOTE - If you add arguments to event_split to make more states,
        be sure to update this function
        args:
            df: input dataframe
    """
    df_out = pd.DataFrame()

    # NOTE - If you add arguments to event_split to make more stats,
    # be sure to update this function

    for index, row in df_in.iterrows():
        dates = row['dates'] # Get event dates
        intensity = row['tmax_tntensity'] # Get intensity for each day
        tmax = row['tmax'] # Get tmax for each day
        ID_HDC_G0 = row['ID_HDC_G0'] # get city id
        # total_days = row['total_days'] # get total number of tmax days -- CPT 2020.02.23

        df = event_split(dates, ID_HDC_G0, intensity, tmax)# , total_days) #CPT 2020.02.23

        df_out = df_out.append(df)
    
    #write df_out
    df_out = 
    
    #return df_out

In [39]:
def stats_parallel(df_in):

    # print current process
    print(str(mp.current_process()))

#     # Open the GHS-ID List with GeoPANDAS read_file
#     ghs_ids_fn = 'GHS-UCSB-IDS.csv'
#     ghs_ids_df = pd.read_csv(DATA_INTERIM+ghs_ids_fn)

#     # Get year for arg for temp_event function
#     #year = fn.split('GHS-Tmax-DAILY_')[1].split('.csv')[0]
#     year = fn.split(FN_IN)[1].split('.csv')[0] # Updated 2020.02.19 CPT
#     print(year)

#     # read csv as a data array
#     temp_xr_da = csv_to_xr(fn, time_dim, space_dim)

#     # data array to tmax events, out as df
#     df_days = tmax_days(temp_xr_da, Tthresh)

#     # tmax events stats, out as df
#     df_out = tmax_stats(df_days)

#     # merge to get countries
#     ghs_ids_df_out = ghs_ids_df.merge(df_out, on='ID_HDC_G0', how = 'inner')
#     # write it all out
#     ghs_ids_df_out.to_csv(DIR_OUT+FN_OUT+year+'.csv')

#     print(year, 'SAVED!')

In [ ]:
# need to create a temp folder
# get length of folder
# have legnth be temp naem

In [40]:
def parallel_loop(function, file_list, cpu_num):
    """  
    """ 
    start = time.time()
    pool = Pool(processes = cpu_num)
    pool.map(function, file_list)
    pool.close()

    end = time.time()
    print(end-start)

In [41]:
# Execute code
print('STARTING LOOP')
parallel_loop(stats_parallel, list_df, 4)
print(Tthresh)
print('ENDING LOOP')

STARTING LOOP
<ForkProcess(ForkPoolWorker-13, started daemon)>
<ForkProcess(ForkPoolWorker-14, started daemon)>
<ForkProcess(ForkPoolWorker-15, started daemon)>
<ForkProcess(ForkPoolWorker-13, started daemon)>
<ForkProcess(ForkPoolWorker-16, started daemon)>
4.783663272857666
40.6
ENDING LOOP


In [42]:
import os

In [50]:
print(os.getcwd())
os.system("cd ../") 
print(os.getcwd())

/home/cascade/projects/UrbanHeat/Notebooks/Cascade_Scratch
/home/cascade/projects/UrbanHeat/Notebooks/Cascade_Scratch


In [52]:
os.system('ls')

0